In [10]:
import torch
import warnings
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from datasets.cityscapes import CityScapes
from tqdm import tqdm

from models.deeplabv2.deeplabv2 import get_deeplab_v2
from utils import fast_hist

warnings.filterwarnings("ignore")

## Training - Single Channel

In [9]:
def train():
    # Set the device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    device = torch.device('cpu')
    # Load the model
    model = get_deeplab_v2(root_dir='models/deeplabv2').to(device)

    # Load the dataset
    dataset = CityScapes(root_dir='datasets/Cityspaces', split='val', mode='single', label_raw=False)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    # Set the model to training mode
    model.train()
    # Train the model
    for epoch in range(10):
        for i, (image, label) in tqdm(enumerate(dataloader)):
            image = image.to(device)
            label = label.to(device)
            # Forward pass
            output = model(image)[0]
            # output shape: (1, 19, 128, 256)
            # label shape: (1, 128, 256)
            
            # Calculate the loss
            loss = criterion(output, label)
            # Zero the gradients
            optimizer.zero_grad()
            # Backward pass
            loss.backward()
            # Update the weights
            optimizer.step()

train()    


Deeplab pretraining loading...


0it [00:00, ?it/s]


IndexError: Target 33 is out of bounds.

In [3]:
torch.cuda.is_available()

True

In [4]:
import torch
print(torch.backends.cudnn.version())

8902


In [5]:
import torch
print(torch.cuda.is_available())
print(torch.__config__.show())

True
PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.3.6 (Git Hash 86e6af5974177e513fd3fee58425e1063e7f1361)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_